In [201]:
"""
Mohammad saeed ibrahim dallash
automated-Snake using openCV
"""
import cv2
import numpy as np
import math
import random
from matplotlib import pyplot as plt

In [202]:
lines = []

size = 50
for i in range (int(size/2 -5) ,int(size/2 + 2)):
    lines.append([int (size/2),i])


GG = False

score = 0
state = 'd'

In [203]:
def move(lines):
    last = lines[-1]
    
    lines.pop(0)

    if state == 's':
        lines.append([last[0]+1, last[1]]) 
    elif state == 'w':
        lines.append([last[0]-1, last[1]]) 
    elif state == 'd':
        lines.append([last[0], last[1]+1]) 
    elif state == 'a':
        lines.append([last[0], last[1]-1]) 

    return lines

In [204]:
def cost_left_right(lines):
    pos = lines[-1][1]
    cost = 0
    for x in lines:
        if x[1] > pos:
            cost+=1
        elif x[1] < pos:
            cost-=1

    return cost

In [205]:
def cost_up_down(lines):
    pos = lines[-1][0]
    cost = 0
    for x in lines:
        if x[0] > pos:
            cost+=1
        elif x[0] < pos:
            cost-=1

    return cost

In [206]:
def decWS(lines,state):
    global score
    if  lines[-1] ==  [0,0] or lines[-1] == [size-1,0]:
        state = 'd' 
    elif  lines[-1] ==[ 0,size-1] or lines[-1] == [size-1,size-1]:
        state = 'a' 
    elif cost_left_right(lines) >= 0:
        if lines[-1][1]==0:
            state = 'd'
            if ([lines[-1][0], lines[-1][1] +1] in lines):
                GG = True
        else:
            state = 'a' 
            if ([lines[-1][0], lines[-1][1] -1] in lines):
                GG = True
    else:
        if lines[-1][1]==size-1:
            state = 'a' 
            if ([lines[-1][0], lines[-1][1] -1] in lines):
                GG = True 
        else:
            state = 'd'
            if ([lines[-1][0], lines[-1][1] +1] in lines):
                GG = True

    return lines,state

In [207]:
def decAD(lines,state):
    
    global score
    global GG
    if  lines[-1] == [0 ,size-1] or lines[-1] == [0,0]:
        state = 's' 
    elif  lines[-1] == [size-1 ,size-1] or lines[-1] == [size-1 ,0]:
        state = 'w'

    elif cost_up_down(lines) >= 0 and lines[-1][0] != 0:
        if lines[-1][0]==0:
            state = 's'
            if ([lines[-1][0]+1, lines[-1][1] ] in lines):
                GG = True
        else :
            state = 'w'
            if ([lines[-1][0]-1, lines[-1][1] ] in lines):
                GG = True
    else:
        if lines[-1][0]==size-1:
            state = 'w'
            if ([lines[-1][0]-1, lines[-1][1] ] in lines):
                GG = True
        else :
            state = 's'
            if ([lines[-1][0]+1, lines[-1][1] ] in lines):
                GG = True

    return lines,state

In [208]:
def decide(lines,state,ball):
    global score
        
    if  lines[-1][0] ==ball[0] and lines[-1][1] != ball[1]: #have the same y
        if lines[-1][1] < ball[1]:
            state = 'd'
        else:
            state = 'a'

    if  lines[-1][1] == ball[1] and lines[-1][0]!=ball[0]: #have the same x
        if lines[-1][0] > ball[0]:
            state = 'w'

        else:
            state = 's'

    if (lines[-1][0] == size-1 or ([lines[-1][0]+1, lines[-1][1] ] in lines)) and state == 's':##     
        lines,state = decWS(lines,state)

    if (lines[-1][0] == 0 or ([lines[-1][0]-1, lines[-1][1]] in lines)) and state == 'w':##
        lines,state = decWS(lines,state) 


    if (lines[-1][1] == size-1 or ([lines[-1][0], lines[-1][1]+1 ] in lines)) and  state == 'd':
        lines,state = decAD(lines,state)

    if (lines[-1][1] == 0 or ([ lines[-1][0], lines[-1][1]-1] in lines)) and state == 'a':
        lines,state = decAD(lines,state)
    
    
    if lines[-1] == ball:
        score+=1
        for i in range(5):
                lines.append(lines[-1])
        ball = [random.randint(0, size-1), random.randint(0, size-1)] 
        while ball in lines:
                ball = [random.randint(0, size-1), random.randint(0, size-1)] 


    return state,ball

In [209]:

def gameOver(lines):
    c=0
    lasti = lines[-1][1]
    lastj = lines[-1][0]
    for j in range (max(lastj-1,0),min (lastj+2,size-1)):
        for i in range (max(lasti-1,0),min (lasti+2,size-1)):
            if  ([j,i] in lines):
                c+=1

    if c==9 :
        return True 
    else:
        return False  

In [210]:
ball = [random.randint(0, size-1), random.randint(0, size-1)] 
while ball in lines:
        ball = [random.randint(0, size-1), random.randint(0, size-1)] 

cv2.namedWindow('img')

while True:

    img = np.zeros( (size,size,1),np.uint8)

    for x in lines:
        img[x[0]][x[1]] = 255
     
    state,ball = decide(lines,state,ball)
    lines = move(lines)
  
    img[ball[0]][ball[1]] = 255

   
    resized = cv2.resize(img, (500,500), interpolation = cv2.INTER_AREA)
    cv2.putText(resized, 'Score '+ str(score),(200,50),5 ,.8,(255,0,0),1)
    cv2.imshow('img',resized)

    k = cv2.waitKey(5)
    if k==ord('q'):
        break

    GG = (GG or gameOver(lines))

    if GG == True:
        break
    
if  GG==True:
    img = np.zeros( (500,500,1),np.uint8)
    cv2.putText(img, 'Game Over',(170,190),5 ,1.2,(255,0,0),1)
    cv2.imshow('img',img)
    k = cv2.waitKey(0)
cv2.destroyAllWindows()